# Solution: Creating the Connected Agent

## Import the libraries

In [7]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import MessageRole, ConnectedAgentTool

## Create the Project Client

In [8]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Run Agent helper function

In [9]:
def run_agent(user_input, thread_id, agent_id):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread_id,  # ID of the thread to which the message belongs
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread_id, agent_id=agent_id)
    print(f"Run finished with status: {run.status}")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread_id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")

## Setup the Connected Agent Tools

In [10]:
# Get the agent by its ID
weather_agent = project_client.agents.get_agent("asst_0NHYVkCkHZsP66inhvkiqehu")
itinerary_agent = project_client.agents.get_agent("asst_hxizwraO7QEbWOGOYh9wicYS")
travel_product_agent = project_client.agents.get_agent("asst_OgzIT5V3gjmYHicCUdWZpboK")
restaurant_agent = project_client.agents.get_agent("asst_wQLX9hQhObUbFnV1qKnECbpH")

weather_connected_agent = ConnectedAgentTool(
        id=weather_agent.id, name="get_weather", description="Searches for information about weather."
    )
itinerary_connected_agent = ConnectedAgentTool(
        id=itinerary_agent.id, name="create_itinerary", description="Generates itineraries for travel."
    )
travel_product_connected_agent = ConnectedAgentTool(
        id=travel_product_agent.id, name="get_travel_products", description="Retrieves travel product information."
    )
restaurant_connected_agent = ConnectedAgentTool(
        id=restaurant_agent.id, name="get_restaurant_recommendations", description="Provides restaurant recommendations."
    )

## Create the Connected Agent

In [11]:
# Create an agent with the file search tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="travel-agent",  # Name of the agent
    instructions="""
        You are an agent that provides travel information. 
        Do not answer questions on your own. 
        Your role is to be an orchestrator who will call the appropriate functions provided to you. 
        Each function that you have available is an agent that can accomplish a specific task. 
        
        Here are descriptions of the tools you have available:
            - Weather Agent: An agent that provides weather information.
            - Itinerary Agent: An agent that creates travel itineraries.
            - Travel Product Agent: An agent that retrieves travel product information.
            - Restaurant Agent: An agent that provides restaurant recommendations.

        Use the tools to gather information and provide a comprehensive response to the user.
        If you are unable to answer the question, please respond with "I don't know".
    """,  # Instructions for the agent
    tools=weather_connected_agent.definitions 
    + itinerary_connected_agent.definitions 
    + travel_product_connected_agent.definitions 
    + restaurant_connected_agent.definitions,  # Tools available to the agent
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_rDsVr48FetZFKsh3kgoLCCyI


## Create the Thread

In [12]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_zkNyx1NOvdr96a6R7MyYtSw0


## Run the agent

In [13]:
# Example user input for the agent
user_input = "What is the weather in Paris?"  # Example user input
run_agent(user_input, thread_id=thread.id, agent_id=agent.id)  # Run the agent with the user input

Created message, ID: msg_56TugXKwN3cKKFLEqc1SLq5j
Run finished with status: completed
Agent Response: The current weather in Paris includes heavy rain with temperatures around 20°C (68°F). The precipitation is expected to reach up to 32mm throughout the day with a 100% chance of rain. Winds are blowing from the southwest at 18km/h, and humidity is quite high at 87%【3:1†source】【3:2†source】.


In [14]:
# Example user input for the agent
user_input = """Create an itinerary for Paris given this weather information:
Currently, in Paris, it is partly cloudy with a temperature of 68°F (20°C). 
The forecast for later today predicts heavy downpours and frequent lightning with afternoon thundershowers. 
The high temperature will reach 69°F (21°C), with a 100% chance of rain
"""

run_agent(user_input, thread_id=thread.id, agent_id=agent.id)  # Run the agent with the user input

Created message, ID: msg_hOIvgbisRkaQfYvPouVYPxiL
Run finished with status: completed
Agent Response: Given the weather information, here's an enjoyable itinerary for Paris that accommodates both the partly cloudy morning and the rainy afternoon:

### Morning (Partly Cloudy)
1. **7:30 AM - 9:00 AM: Breakfast at a Parisian Café**
   - Start your day with a traditional French breakfast at a cozy café like Café de Flore. Enjoy coffee, croissants, and fresh fruit.

2. **9:15 AM - 11:00 AM: Eiffel Tower Visit**
   - Take advantage of the clear morning to visit the Eiffel Tower. Enjoy the panoramic views of Paris from the top.

3. **11:30 AM - 12:30 PM: Walk along the Seine River**
   - Enjoy a leisurely stroll along the Seine River, taking in the historic bridges and picturesque scenery.

### Midday (Prepare for Changing Weather)
4. **12:45 PM - 2:00 PM: Lunch at a Covered Market**
   - Head to Marché des Enfants Rouges, a famous covered market, for a variety of French delicacies while stay

In [15]:
# Example user input for the agent
user_input = "What suitcases do you have available?"  # Example user input
run_agent(user_input, thread_id=thread.id, agent_id=agent.id)  # Run the agent with the user input

Created message, ID: msg_v8STHkEIGgy6mne6MYX62vJO
Run finished with status: completed
Agent Response: Here are some of the available suitcases with their details:

1. **Nomad Traveler Suitcase**
   - **Price**: $275
   - **Material**: High-impact ABS plastic
   - **Features**:
     - Expandable design for extra packing space
     - 360-degree spinner wheels
     - TSA-approved combination lock
     - Multiple interior and exterior compartments
     - Lightweight (7 lbs)
     - Water-resistant exterior
   - **Dimensions**: 23 inches x 15 inches x 9 inches
   - **Capacity**: 48 liters
   - **Warranty**: 5-year limited warranty
   - **Color Options**: Black, Green, Purple, Silver  

2. **Globetrotter Elite Suitcase**
   - **Price**: $299
   - **Material**: High-grade polycarbonate
   - **Features**:
     - Expandable design for additional packing space
     - 360-degree spinner wheels
     - TSA-approved combination lock
     - Multiple interior and exterior compartments
     - Lightweigh

In [16]:
# Example user input for the agent
user_input = "Provide 3 recommendations for Pizza in Paris?"  # Example user input
run_agent(user_input, thread_id=thread.id, agent_id=agent.id)  # Run the agent with the user input

Created message, ID: msg_10jBBQqksvOxUTrNKO7TC53Q
Run finished with status: completed
Agent Response: Here are three recommended pizza restaurants in Paris:

1. **Le Weekend Bar Brasserie**
   - **Address**: 19 Rue Tronchet, 75008 Paris, France

2. **Oscar**
   - **Address**: Rue de Chaillot, 75016 Paris, France

3. **Nachos**
   - **Address**: 101 Porte Berger, Forum Des Halles, 75001 Paris, France

These options offer a variety of pizza choices along with other dining selections. Enjoy your meal in Paris!


In [17]:
# Example user input for the agent
user_input = """
    Provide me 3 reviews of the Pizzeria Arrivederci in Paris.    
"""  # Example user input
run_agent(user_input, thread_id=thread.id, agent_id=agent.id) 

Created message, ID: msg_NK8gq3a4lDtFhsyKyHdYAp3U
Run finished with status: completed
Agent Response: Here are three reviews for **Pizzeria Arrivederci** in Paris:

1. **Great pizza in Paris**
   - **Rating**: ⭐⭐⭐⭐⭐
   - **Review**: "Fantastic pizzas and brilliant atmosphere, we had a great evening. We have two young kids and it worked well for them too."
   - **User**: clarefB7005CD
   - [Link to Review](https://www.tripadvisor.com/ShowUserReviews-g187147-d12714552-r1016682668-Reviews-Pizzeria_Arrivederci-Paris_Ile_de_France.html?m=66827#review1016682668)

2. **Wauw..!**
   - **Rating**: ⭐⭐⭐⭐⭐
   - **Review**: "Food was a 10/10, service was so good!! Very nice people that make you feel very welcome. It’s not rushed which we liked a lot and it was just a really nice authentic experience :)"
   - **User**: EsterJelena from Amsterdam, North Holland Province
   - [Link to Review](https://www.tripadvisor.com/ShowUserReviews-g187147-d12714552-r1016536196-Reviews-Pizzeria_Arrivederci-Paris_I